In [3]:
import requests
import time
import pandas as pd
import re
import json
from bs4 import BeautifulSoup
from random import uniform
from datetime import datetime
from tqdm import tqdm
from IPython.display import clear_output

## Scrapping of one game :

In [4]:
urls = 'https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/'

page = requests.get(urls, headers = {"Accept-Language": "en-US"})
bs = BeautifulSoup(page.text, 'html.parser')

# Find name
name_block = bs.find('div','apphub_AppName')
name = name_block.contents[0]
print(name)

# Find genres
genreblock = bs.find('div','details_block')
title = genreblock.findAll('a')

genres = ['Simulation', 'Sports', 'Action', 'Adventure', 'RPG', 'Strategy', 'Sports', 'Racing', 'Indie',
         'Free to Play', 'Massively Multiplayer', 'Casual', 'Early Access']

genre = []

for g in title:
    if g.contents[0] in genres:
        genre.append(g.contents[0])
        print(g.contents[0])

# Find rating
rating_block = bs.find('div', 'summary_section')
rating_span = rating_block.findAll('span','game_review_summary')
rating = rating_span[0].contents[0]
print(rating)


# Price
price = None

    # Case 1 : Discount
tr = bs.find('div', 'discount_original_price')
if tr:
    price = tr.text.strip()

    # Case 2 : No Discount
if not price:
    no_disc = bs.find('div', 'game_purchase_price price')
    if no_disc:
        price = no_disc.text.strip()

# In-App Purchases
inAppPurchase = False

cat_block = bs.find('div', 'game_area_features_list_ctn')
categor = cat_block.findAll('a')
for cat in categor:
    ok = cat.find('div', 'label')
    if 'In-App Purchases' in ok.contents[0]:
        inAppPurchase = True
print(inAppPurchase)

# DLC
has_DLC = False

dlc = bs.find('div', 'game_area_dlc_section')
if dlc:
    has_DLC = True
print(has_DLC)

# Release Date
release_date = 'N/A'

release_date_block = bs.find('div', class_='release_date')
if release_date_block:
    date_div = release_date_block.find('div', class_='date')
    release_date = date_div.text.strip() if date_div else 'N/A'
    release_date_obj = datetime.strptime(release_date, "%d %b, %Y")
print(release_date_obj)

# Multiplayer
tags = []
is_multiplayer = False

tag_block = bs.find_all('a', class_='app_tag')
tags = [tag.text.strip() for tag in tag_block if tag.text.strip()]
is_multiplayer = any("multiplayer" in tag.lower() for tag in tags)
print(is_multiplayer)

Counter-Strike 2
Action
Very Positive
True
True
2012-08-21 00:00:00
True


## Scraping most sold games

Each page contains 25 games.

In [5]:
number_of_pages_scraped = 280

In [6]:
links = []

for i in tqdm(range(1, number_of_pages_scraped + 1), desc="Scraping pages") :
    url = 'https://store.steampowered.com/search/?sort_by=&sort_order=0&filter=topsellers&supportedlang=english&page=' + str(i)

    page = requests.get(url, headers = {"Accept-Language": "en-US"})
    bs = BeautifulSoup(page.text, 'html.parser')
    games = bs.find('div', 'search_results')
    game_list = games.findAll('a')

    for j in range(25):
        links.append(game_list[j]['href'])
    randsleep = uniform(0.0,0.7)
    time.sleep(randsleep)

Scraping pages:   0%|          | 0/280 [00:00<?, ?it/s]

Scraping pages: 100%|██████████| 280/280 [05:00<00:00,  1.07s/it]


In [7]:
df = pd.DataFrame(columns=['gameId','Name', 'Price', 'Genres', 'Rating', 'hasDLC', 'inAppPurchases', 'Release Date', 'Multiplayer'])

headers = {"Accept-Language": "en-US"}
cookies = {
    'birthtime': '568022401',
    'lastagecheckage': '1-January-1988'
}

for i in tqdm(range(number_of_pages_scraped*25), desc="Scraping games") :
    try:
        page = requests.get(links[i], headers=headers, cookies=cookies)
        bs = BeautifulSoup(page.text, 'html.parser')

        gameid = links[i].split('/')[4]

        # Name
        name_block = bs.find('div','apphub_AppName')
        name = name_block.contents[0] if name_block else "N/A"

        # Find genres
        genreblock = bs.find('div','details_block')
        title = genreblock.findAll('a')

        genres = ['Simulation', 'Sports', 'Action', 'Adventure', 'RPG', 'Strategy', 'Sports', 'Racing', 'Indie',
                'Free to Play', 'Massively Multiplayer', 'Casual', 'Early Access']
        genre = []

        for g in title:
            if g.contents[0] in genres:
                genre.append(g.contents[0])

        # Find rating
        rating = "N/A"

        rating_block = bs.find('div', 'summary_section')
        if rating_block:
            rating_span = rating_block.findAll('span','game_review_summary')
            rating = rating_span[0].contents[0]
        
        # Price
        price = None
        
            # Case 1 : Discount
        tr = bs.find('div', 'discount_original_price')
        if tr:
            price = tr.text.strip()

            # Case 2 : No Discount
        if not price:
            no_disc = bs.find('div', 'game_purchase_price price')
            if no_disc:
                price = no_disc.text.strip()

        # In-App Purchases
        inAppPurchase = False

        cat_block = bs.find('div', 'game_area_features_list_ctn')
        categor = cat_block.findAll('a')
        for cat in categor:
            ok = cat.find('div', 'label')
            if 'In-App Purchases' in ok.contents[0]:
                inAppPurchase = True

        # DLC
        has_DLC = False

        dlc = bs.find('div', 'game_area_dlc_section')
        if dlc:
            has_DLC = True

        # Release Date
        release_date = 'N/A'

        release_date_block = bs.find('div', class_='release_date')
        if release_date_block:
            date_div = release_date_block.find('div', class_='date')
            release_date = date_div.text.strip() if date_div else 'N/A'
            release_date_obj = datetime.strptime(release_date, "%d %b, %Y")

        # Multiplayer
        tags = []
        is_multiplayer = False

        tag_block = bs.find_all('a', class_='app_tag')
        tags = [tag.text.strip() for tag in tag_block if tag.text.strip()]
        is_multiplayer = any("multiplayer" in tag.lower() for tag in tags)

        # Add row
        row = {'gameId':[gameid],'Name':[name], 'Genres':[genre], 'Price':[price],'Rating':[rating], 'hasDLC':[has_DLC], 
            'inAppPurchases': [inAppPurchase], 'Release Date': [release_date_obj], 'Multiplayer': [is_multiplayer]}
        df = pd.concat([df,pd.DataFrame(row)])

        # Break
        randsleep = uniform(0.0,0.6)
        time.sleep(randsleep)

        # Break every 100 requests
        if (i + 1) % 100 == 0:
            time.sleep(uniform(2, 5))

    except Exception as e:
        print(f"Erreur sur le jeu {links[i]} : {e}")

df = df.reset_index(drop=True)

Scraping games:   0%|          | 0/7000 [00:00<?, ?it/s]

C:\Users\maxen\AppData\Local\Temp\ipykernel_17012\113701633.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,pd.DataFrame(row)])
Scraping games:   0%|          | 4/7000 [00:03<1:28:57,  1.31it/s]

Erreur sur le jeu https://store.steampowered.com/app/1675200/Steam_Deck/?snr=1_7_7_7000_150_1 : 'NoneType' object has no attribute 'findAll'


Scraping games:   2%|▏         | 120/7000 [01:44<1:38:45,  1.16it/s]

Erreur sur le jeu https://store.steampowered.com/app/1696780/Steam_Deck_Docking_Station/?snr=1_7_7_7000_150_5 : 'NoneType' object has no attribute 'findAll'


...

Scraping games:  98%|█████████▊| 6838/7000 [1:45:31<02:31,  1.07it/s]

Erreur sur le jeu https://store.steampowered.com/app/2248940/Read_Clock_Time_Soundtrack/?snr=1_7_7_7000_150_274 : 'NoneType' object has no attribute 'findAll'


Scraping games:  98%|█████████▊| 6850/7000 [1:45:38<01:28,  1.69it/s]

Erreur sur le jeu https://store.steampowered.com/app/1877810/Iron_Lung_Soundtrack/?snr=1_7_7_7000_150_274 : 'NoneType' object has no attribute 'findAll'


Scraping games: 100%|██████████| 7000/7000 [1:54:08<00:00,  1.02it/s]   


In [8]:
df.head()

,gameId,Name,Price,Genres,Rating,hasDLC,inAppPurchases,Release Date,Multiplayer
0,1903340,Clair Obscur: Expedition 33,"67,48€","[Action, RPG]",Very Positive,True,False,2025-04-24,False
1,2993780,FANTASY LIFE i: The Girl Who Steals Time,"59,99€",[RPG],Very Positive,False,False,2025-05-21,True
2,730,Counter-Strike 2,Free To Play,[Action],Very Positive,True,True,2012-08-21,True
3,1091500,Cyberpunk 2077,"59,99€",[RPG],Very Positive,True,False,2020-12-09,False
4,2488370,Cash Cleaner Simulator,"14,79€","[Casual, Indie, Simulation]",Very Positive,True,False,2025-05-08,False


In [9]:
df.to_csv('./gamelist4.csv')
df.to_excel('./gamelist4.xlsx')